In [82]:
import xlwings as xw
import pandas as pd
from nsepython import *
from datetime import datetime, timedelta
import math

In [83]:
def gethighofadate(symbol, dt):
    start_date = (datetime.today() - timedelta(days=7)).strftime("%m/%d/%Y")
    end_date = datetime.now().strftime("%m/%d/%Y")
    df = pd.DataFrame(index_history(symbol,start_date,end_date))
    dt = dt.strftime("%d %b %Y")
    filt = (df['HistoricalDate'] == dt)
    try:
        result = df.loc[filt,'HIGH'].values[0]
        return float(result)
    except IndexError:
        return "NA"
    except ValueError as ve:
        return ve
def getlowofadate(symbol, dt):
    start_date = (datetime.today() - timedelta(days=7)).strftime("%m/%d/%Y")
    end_date = datetime.now().strftime("%m/%d/%Y")
    df = pd.DataFrame(index_history(symbol,start_date,end_date))
    dt = dt.strftime("%d %b %Y")
    filt = (df['HistoricalDate'] == dt)
    try:
        result = df.loc[filt,'LOW'].values[0]
        return float(result)
    except IndexError:
        return "NA"
def derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice):
    optionType = "CE"
    start_date = start_date.strftime("%d-%m-%Y")
    end_date = end_date.strftime("%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    try:
        df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
        result = (df['FH_TRADE_LOW_PRICE'].min())
        return float(result)
    except:
        return "NA"
def derivativehistoryputgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,):
    optionType = "PE"
    start_date = start_date.strftime("%d-%m-%Y")
    end_date = end_date.strftime("%d-%m-%Y")
    expiry_date = expiry_date.strftime("%d-%b-%Y")
    try:
        df = pd.DataFrame(derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice,optionType))
        result = (df['FH_TRADE_LOW_PRICE'].max())
        return float(result)
    except:
        return "NA"
def round_up_to_base(x, base=50):
    return x + (base - x) % base
def round_down_to_base(x, base=50):
    return x - (x % base)

In [84]:
# clear data 
wb = xw.Book('Formula R1 input data.xlsx') 
sheet = wb.sheets['R2']
sheet.range("C5:F6").clear_contents()
sheet.range("K5:K6").clear_contents()
sheet.range("B10:F21").clear_contents()
sheet.range("H10:L21").clear_contents()
sheet.range("D24:D25").clear_contents()
sheet.range("J24:J25").clear_contents()

In [85]:
# Filling rows 4,5,6
symbol = "NIFTY 50"
start_date = sheet['C4'].value
end_date = sheet['D4'].value
expiry_date = sheet['J3'].value
instrumentType = "options"
highofstartdate = gethighofadate(symbol,sheet['C4'].value)
sheet['C5'].value= highofstartdate
lowofstartdate = getlowofadate(symbol,sheet['C4'].value)
sheet['C6'].value = lowofstartdate
highofenddate = gethighofadate(symbol,sheet['D4'].value)
sheet['D5'].value= highofenddate
lowofenddate = getlowofadate(symbol,sheet['D4'].value)
sheet['D6'].value = lowofenddate
maxof2days = max(highofstartdate,highofenddate)
sheet['E5'].value = maxof2days
minof2days = min(lowofstartdate,lowofenddate)
sheet['E6'].value = minof2days
bufferhigh = round(maxof2days * 1.0015)
sheet['F5'].value = bufferhigh
bufferlow = round(minof2days * 0.9985)
sheet['F6'].value = bufferlow
callendstrike = round_down_to_base(bufferlow)
sheet['K5'].value = callendstrike
putendstrike = round_up_to_base(bufferhigh)
sheet['K6'].value = putendstrike

In [86]:
# finding CallEntryStrike 1st row
symbol = "NIFTY"
start_date = sheet['C4'].value
end_date = sheet['D4'].value
expiry_date = sheet['J3'].value
instrumentType = "options"
strike = []
premium = []
twodll = []
strike.append(callendstrike)
temppremium = callendstrike * (0.85/100)
premium.append(temppremium)
temptwodll = derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[0])
twodll.append(temptwodll)

{'data': [{'_id': '65ce14fa7e26484fc0b2b279', 'FH_INSTRUMENT': 'OPTIDX', 'FH_SYMBOL': 'NIFTY', 'FH_EXPIRY_DT': '15-Feb-2024', 'FH_STRIKE_PRICE': '21450.00', 'FH_OPTION_TYPE': 'CE', 'FH_MARKET_TYPE': 'N', 'FH_OPENING_PRICE': '472.60', 'FH_TRADE_HIGH_PRICE': '514.25', 'FH_TRADE_LOW_PRICE': '345.45', 'FH_CLOSING_PRICE': '456.75', 'FH_LAST_TRADED_PRICE': '461.25', 'FH_PREV_CLS': '426.70', 'FH_SETTLE_PRICE': '21910.75', 'FH_TOT_TRADED_QTY': '253450', 'FH_TOT_TRADED_VAL': '5546179745.00', 'FH_OPEN_INT': '25950', 'FH_CHANGE_IN_OI': '-26650.00', 'FH_MARKET_LOT': '50', 'FH_TIMESTAMP': '15-Feb-2024', 'TIMESTAMP': '2024-02-14T18:30:00.000Z', 'FH_UNDERLYING_VALUE': 21910.75, 'CALCULATED_PREMIUM_VAL': 109677245}, {'_id': '65ccd6017e26484fc0971caf', 'FH_INSTRUMENT': 'OPTIDX', 'FH_SYMBOL': 'NIFTY', 'FH_EXPIRY_DT': '15-Feb-2024', 'FH_STRIKE_PRICE': '21450.00', 'FH_OPTION_TYPE': 'CE', 'FH_MARKET_TYPE': 'N', 'FH_OPENING_PRICE': '184.35', 'FH_TRADE_HIGH_PRICE': '477.15', 'FH_TRADE_LOW_PRICE': '146.35', '

In [87]:
# Call Entry strke 2....n rows
for i in range(1,9):
    if twodll[-1] >= premium[-1]:
        strike.append(strike[-1]+50)
        premium.append(strike[-1]* (0.85/100))
        twodll.append(derivativehistorycallgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[-1]))
    else:
        break
calldf = pd.DataFrame({"strike":strike,"premium":premium,"twodll":twodll})
calldf['diff']=round(calldf['twodll']-calldf['premium'],2)
print (calldf)
leastdiff = calldf.loc[calldf['diff']>0,'diff'].min()
leastdiff = round(leastdiff,2)
print (leastdiff)
sheet['B10'].value = calldf    
sheet['F21'].value = leastdiff
if math.isnan(leastdiff):
    callentrystrike = 'NA'
    calltwodll = 'NA'
else:
    filt = (calldf['diff'] == leastdiff)
    callentrystrike = calldf.loc[filt,'strike'].values[0]
    calltwodll = calldf.loc[filt,'twodll'].values[0]
sheet['D24'].value = callentrystrike
sheet['D25'].value = calltwodll


   strike  premium  twodll   diff
0   21450  182.325  146.35 -35.98
nan


In [88]:
# finding PutEntryStrike 1st row
symbol = "NIFTY"
start_date = sheet['C4'].value
end_date = sheet['D4'].value
expiry_date = sheet['J3'].value
instrumentType = "options"
strike = []
premium = []
twodll = []
strike.append(putendstrike)
temppremium = putendstrike * (0.85/100)
premium.append(temppremium)
temptwodll = derivativehistoryputgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[0])
twodll.append(temptwodll)

{'data': [{'_id': '65ce15146add4a917de1abba', 'FH_INSTRUMENT': 'OPTIDX', 'FH_SYMBOL': 'NIFTY', 'FH_EXPIRY_DT': '15-Feb-2024', 'FH_STRIKE_PRICE': '22000.00', 'FH_OPTION_TYPE': 'PE', 'FH_MARKET_TYPE': 'N', 'FH_OPENING_PRICE': '134.65', 'FH_TRADE_HIGH_PRICE': '214.70', 'FH_TRADE_LOW_PRICE': '54.00', 'FH_CLOSING_PRICE': '93.80', 'FH_LAST_TRADED_PRICE': '89.25', 'FH_PREV_CLS': '148.40', 'FH_SETTLE_PRICE': '21910.75', 'FH_TOT_TRADED_QTY': '182637850', 'FH_TOT_TRADED_VAL': '4038026446200.00', 'FH_OPEN_INT': '1845400', 'FH_CHANGE_IN_OI': '515000.00', 'FH_MARKET_LOT': '50', 'FH_TIMESTAMP': '15-Feb-2024', 'TIMESTAMP': '2024-02-14T18:30:00.000Z', 'FH_UNDERLYING_VALUE': 21910.75, 'CALCULATED_PREMIUM_VAL': 19993746200}, {'_id': '65ccd62d6add4a917daf1c02', 'FH_INSTRUMENT': 'OPTIDX', 'FH_SYMBOL': 'NIFTY', 'FH_EXPIRY_DT': '15-Feb-2024', 'FH_STRIKE_PRICE': '22000.00', 'FH_OPTION_TYPE': 'PE', 'FH_MARKET_TYPE': 'N', 'FH_OPENING_PRICE': '400.00', 'FH_TRADE_HIGH_PRICE': '480.00', 'FH_TRADE_LOW_PRICE': '122

In [89]:
# Put Entry strke 2....n rows
for i in range(1,9):
    if twodll[-1] >= premium[-1]:
        strike.append(strike[-1]-50)
        premium.append(strike[-1]* (0.85/100))
        twodll.append(derivativehistoryputgetlowvalue(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice=strike[-1]))
    else:
        break
putdf = pd.DataFrame({"strike":strike,"premium":premium,"twodll":twodll})
putdf['diff']=round(putdf['twodll']-putdf['premium'],2)
print (calldf)
leastdiff = putdf.loc[putdf['diff']>0,'diff'].min()
leastdiff = round(leastdiff,2)
print (leastdiff)
sheet['H10'].value = putdf    
sheet['L21'].value = leastdiff
if math.isnan(leastdiff):
    putentrystrike = 'NA'
    puttwodll = 'NA'
else:
    filt = (putdf['diff'] == leastdiff)
    putentrystrike = putdf.loc[filt,'strike'].values[0]
    puttwodll = putdf.loc[filt,'twodll'].values[0]

sheet['J24'].value = putentrystrike
sheet['J25'].value = puttwodll

   strike  premium  twodll   diff
0   21450  182.325  146.35 -35.98
nan
